In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

import cufflinks as cf
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)
cf.go_offline()

In [33]:
train_df = pd.read_csv(r'C:\Users\wwwKa\OneDrive\Desktop\Kaggle 2\applications-of-machine-learning-prml-hackathon-2\train')
test_df = pd.read_csv(r'C:\Users\wwwKa\OneDrive\Desktop\Kaggle 2\applications-of-machine-learning-prml-hackathon-2\test')
train_df[['c1', 'c2', 'c3', 'c4', 'c5', 'c6']] = train_df['Prediction'].str.split(',', expand=True)
train_df.drop('Prediction', axis=1, inplace=True)
cols = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6']
train_df1 = train_df[cols]
for i in cols:
    train_df1[i] = train_df1[i].astype(int)
    train_df[i] = train_df[i].astype(int)
for i in cols:
    print(f'{i} : {train_df1[i].sum()}')

c1 : 10676
c2 : 1087
c3 : 5905
c4 : 321
c5 : 5523
c6 : 967


In [34]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import re, string


In [35]:
comment_len = train_df['comment_text'].str.len()
comment_len.mean(), comment_len.std(), comment_len.max()

(394.5207164147042, 591.9814508693155, 5000)

In [36]:
comment_len.iplot(kind='hist')

In [37]:
label = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6']
train_df['none'] = 1 - train_df[label].max(axis=1)
train_df.head()

,id,comment_text,c1,c2,c3,c4,c5,c6,none
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,1
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,1
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,1
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,1
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,1


In [38]:
train_df.shape, test_df.shape

((111723, 9), (47848, 2))

In [39]:
train_df['comment_text'].isna().sum()


0

In [40]:
np.where(comment_len==0)

(array([], dtype=int64),)

In [41]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')

def tokenize(s):
    return re_tok.sub(r' \1 ', s).split()

In [42]:
tfidf_vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize)
train_term_doc = tfidf_vec.fit_transform(train_df['comment_text'])
test_trem_doc = tfidf_vec.transform(test_df['comment_text'])

In [43]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [44]:
x = train_term_doc
test_x = test_trem_doc

In [45]:
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=True, solver='liblinear')
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [46]:
preds = np.zeros((len(test_df), len(label)))

for i, j in enumerate(label):
    print('fit', j)
    m,r = get_mdl(train_df[j])
    preds[:,i] = m.predict(test_x.multiply(r))

fit c1
fit c2
fit c3
fit c4
fit c5
fit c6


In [21]:
newframe = pd.DataFrame(preds, columns=['p1', 'p2', 'p3', 'p4', 'p5', 'p6'])

In [23]:
t = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6']

In [24]:
for i in t:
    newframe[i] = newframe[i].astype(int)


In [26]:
for i in t:
    newframe[i] = newframe[i].astype(str)

In [27]:
newframe['Prediction'] = newframe['p1'] +','+ newframe['p2'] +','+ newframe['p3'] +','+ newframe['p4'] +','+ newframe['p5'] +','+ newframe['p6']

In [29]:
for i in t:
    newframe.drop([i], axis=1, inplace=True)

In [30]:
newframe.to_csv('file4.csv')